# The Movies Dataset TF-IDF Content-Based Recommendation

##Понимание данных:

### Набор данных

Дан набор данных, содержащий информацию о 45,000 фильмах, выпущенных до июля 2017 года. Набор данных представлен на ресурсе Kaggle по ссылке https://www.kaggle.com/rounakbanik/the-movies-dataset
где представлено следующее описание составляющих файлов (выполнен перевод на русский язык):

`movies_metadata.csv:` Основной файл метаданных фильмов. Содержит информацию о 45 000 фильмов, представленных в наборе данных Full MovieLens. В таблицы представлены плакаты, фоны, бюджет, доход, даты выпуска, языки, страны-производители и компании.

`keywords.csv:` Содержит ключевые слова сюжета для наших фильмов MovieLens. Доступен в виде строкового объекта JSON.

`credits.csv:` Состоит из информации об актерах и съемках всех наших фильмов. Доступен в виде строкового объекта JSON. 

`links.csv:` Файл содержит идентификаторы TMDB и IMDB всех фильмов, представленных в наборе данных Full MovieLens.

`links_small.csv:` Содержит идентификаторы TMDB и IMDB небольшого подмножества из 9000 фильмов полного набора данных.

`ratings_small.csv:` Подмножество 100 000 оценок от 700 пользователей на 9 000 фильмов.

`ratings.csv `- файл, содержащий полный список оценок, выставленных пользователями фильмам



Рассмотрим подробнее две таблицы:

*   `movies_metadata.csv `
*   `ratings.csv `

In [1]:
import pandas as pd
import numpy as np

In [2]:
! pip install --upgrade --no-cache-dir gdown

You should consider upgrading via the 'C:\Users\milen\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
#!gdown 14LeiPV598IHba4rGi07mhfC7O2Iw-NoL

In [4]:
#!gdown 17nemNXYD8D_rNdWSgVbcBrrNAozxTWgg

Начинаем с анализа датафрейма movies_metadata

## Задание 1

In [260]:
metadata = pd.read_csv("movies_metadata.csv")

C:\Users\milen\AppData\Local\Temp\ipykernel_16056\1912714352.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("movies_metadata.csv")


In [268]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Удалите из датафрейма metadata строки, в которых отсутствует описание. Обратите внимание, что у некоторых фильмов формально описание есть, но там написано No overview found, No overview, No movie overview available, Released. Такие строки тоже нужно удалить. 

In [269]:
metadata = metadata[metadata['overview'].notnull()]


In [270]:
no_overview_string = ['No overview found','No Overview', 'No movie overview available', 'Released']

def filter_inapropriate_overviews(overview):
    for no_overview in no_overview_string:
      if no_overview.lower() in overview.lower():
        return None
    if len(overview.split()) <= 1:
      return None
    return overview

In [271]:
metadata['overview'] = metadata.apply(lambda row: filter_inapropriate_overviews(row['overview']), axis=1)
metadata = metadata[metadata['overview'].notnull()]

C:\Users\milen\AppData\Local\Temp\ipykernel_16056\3804731966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata['overview'] = metadata.apply(lambda row: filter_inapropriate_overviews(row['overview']), axis=1)


In [272]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43856 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  43856 non-null  object 
 1   belongs_to_collection  4360 non-null   object 
 2   budget                 43856 non-null  object 
 3   genres                 43856 non-null  object 
 4   homepage               7674 non-null   object 
 5   id                     43856 non-null  object 
 6   imdb_id                43841 non-null  object 
 7   original_language      43846 non-null  object 
 8   original_title         43856 non-null  object 
 9   overview               43856 non-null  object 
 10  popularity             43853 non-null  object 
 11  poster_path            43520 non-null  object 
 12  production_companies   43853 non-null  object 
 13  production_countries   43853 non-null  object 
 14  release_date           43783 non-null  object 
 15  re

## Задание 2

Оставьте в датафрейме столбцы `'id', 'imdb_id', 'overview', 'title'`. Выведите 10 случайных строк.

In [273]:
metadata = metadata[['id', 'imdb_id', 'overview', 'title']]
metadata['id'] = metadata.apply(lambda row: int(row['id']), axis=1)
display(metadata.sample(10))

,id,imdb_id,overview,title
18893,70386,tt1712578,Set in the Channel Islands on the eve of D Day...,The Devil's Rock
11682,52679,tt0019071,A former Imperial Russian general and cousin o...,The Last Command
4101,10833,tt0125022,Max and Page are a brilliant mother/daughter c...,Heartbreakers
10594,253250,tt0040825,"Red Skelton plays Aubrey Filmore, a feather-br...",A Southern Yankee
43662,289269,tt4327752,Jane is a beautiful but troubled American girl...,Lady Bloodfight
16789,22620,tt1258157,"What brings best friends together? For Daphne,...",Scooby-Doo! The Mystery Begins
38814,294368,tt3677412,An investigation into accusations of teenagers...,An Open Secret
45112,72363,tt0348841,"A love triangle between a businessman, his wif...",I'm Staying
31980,128960,tt0054854,Simple story about the man who comes to the be...,The Substitute
28422,186970,tt2852394,A 25 year old young man with dreams of becomin...,Grigris


## Задание 3

In [274]:
ratings = pd.read_csv("ratings.csv")

In [275]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148
6,1,2762,4.5,1425941300
7,1,2918,5.0,1425941593
8,1,2959,4.0,1425941601
9,1,4226,4.0,1425942228


In [276]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


Убедимся в отсутствии пропусков

In [277]:
pd.isnull(ratings).sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Объедините датафреймы `metadata` и `ratings` в один. Обратите внимание, что `'id'` в  `metadata`, этот тот же самый идентификатор, что и `'movie_id'` и `ratings`. Объединять нужно по этому идентификатору (также обратите внимание на его тип данных).

In [278]:
data = pd.merge(metadata, ratings, left_on=['id'], right_on = ['movieId'])

## Задание 4

In [279]:
pd.isnull(data).sum()

id            0
imdb_id      47
overview      0
title         0
userId        0
movieId       0
rating        0
timestamp     0
dtype: int64

Появились ли пропуски в получившемся после объединения датафрейме? Если появились, то ответьте на вопрос "почему?" и удалите строки с пропусками.

Выведите размеры получившегося датафрейма и 10 случайных записей в нём.

In [280]:
data.info()
display(data.sample(10))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11223149 entries, 0 to 11223148
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         int64  
 1   imdb_id    object 
 2   overview   object 
 3   title      object 
 4   userId     int64  
 5   movieId    int64  
 6   rating     float64
 7   timestamp  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 770.6+ MB


,id,imdb_id,overview,title,userId,movieId,rating,timestamp
1810860,1281,tt0118689,Bean works as a caretaker at Britain's formida...,Bean,185167,1281,4.0,1467976646
7203672,919,tt0275230,"In Japan, the vampire-hunter Saya, who is a po...",Blood: The Last Vampire,14495,919,4.0,965369430
1769520,266,tt0057345,Screenwriter Paul Javal's marriage to his wife...,Contempt,83918,266,4.0,834397606
5524393,296,tt0181852,It's been 10 years since John Connor saved Ear...,Terminator 3: Rise of the Machines,35897,296,4.5,1317492457
8540696,6443,tt0386862,As an awkward idealistic high school teacher b...,The Forest for the Trees,259446,6443,3.0,1159832505
5979893,899,tt0009968,Broken Blossoms is an American silent film fro...,Broken Blossoms,203114,899,4.0,1367773862
7536915,953,tt0351283,Zoo animals leave the comforts of man-made hab...,Madagascar,231867,953,2.0,944375121
3626514,955,tt0120755,With computer genius Luther Stickell at his si...,Mission: Impossible II,127217,955,4.5,1386016326
2438137,2109,tt0120812,When Hong Kong Inspector Lee is summoned to Lo...,Rush Hour,124146,2109,1.0,956674369
9038455,6639,tt0484740,"In Colombia just after the Great War, an old m...",Love in the Time of Cholera,207001,6639,4.5,1190913744


## Задание 5

In [19]:
movies = dict()
for _, review in data.drop_duplicates(subset=['movieId']).iterrows():
    movies[review['movieId']] = review['title']

In [20]:
critics = {}
for _, review in data.iterrows():
    if review['userId'] in critics.keys():
        critics[review['userId']][review['movieId']] = review['rating']
    else:
        critics[review['userId']] = {review['movieId']: review['rating']}

Возьмите случайного пользователя (проверьте, чтобы у него было достаточное количество оценок). Сформируйте для этого пользователя рекомендацию методом коллаборативной фильтрации. Оцените качество это рекомендации.


Повторите эти рассчеты для большого числа пользователей и дайте интегральную оценку.

**Будем считать что достаточное количество оценок - 666**



In [21]:
def pick_random_user(df, overview_count = 666):
  user_ratings_count = df.groupby('userId')['userId'].count()
  user_ratings_count = user_ratings_count.loc[lambda x : x >= overview_count]
  return user_ratings_count.sample(1)

In [22]:
def similar_films(critics_dict: dict, person1, person2):
    # создаем пустой список фильмов с названием
    sim_film = []
    # для всех фильмов у person1, если такой фильм оценивался person 2, добавляем этот фильм в список sim_film
    for film in critics_dict[person1].keys():
        if film in critics_dict[person2].keys():
            sim_film.append(film)
    ## your code here

    return sim_film

In [23]:
def compute_euclid(x, y):
    return np.sqrt(sum([ (x_ - y_) ** 2 for x_,y_ in zip(x, y)]))

In [32]:
def euclid_oxana(critics_dict, person1, person2):
    # используем написанную функцию для получения фильмов, оцененных обоими критиками
    sim_films = similar_films(critics_dict, person1, person2)

    # если нет ни одной общей оценки, то есть длина sim_film равна 0 - вернуть 0
    if len(sim_films) == 0:
        return 0

    ## your code here
    x = []
    y = []
    for film in sim_films:
        x.append(critics_dict[person1][film])
        y.append(critics_dict[person2][film])

    # посчитаем евклидово расстояние между критиками по фильмам из sim_films
    # посчитаем похожесть score
    return 1 / (1 + compute_euclid(x, y))

In [25]:
from scipy.stats import pearsonr
# Возвращает коэффициент корреляции Пирсона между person1 и person2
def sim_pearson(critics_dict, person1, person2):
    from math import isnan # это нам пригодиться для обработки деления на 0

    # получить список фильмов, оцененных обоими критиками
    sim_films = similar_films(critics_dict, person1, person2)

    ## your code here

    # если нет ни одной общей оценки, вернуть 0
    if len(sim_films) < 2:
        return 0

    # получим оценки критиков для фильмов из пересечения
    scores1 = []
    scores2 = []
    for film in sim_films:
        scores1.append(critics_dict[person1][film])
        scores2.append(critics_dict[person2][film])

    # посчитаем коэффициент корреляции
    r = pearsonr(scores1, scores2)[0]

    ## end of your code

    # если вдруг было деление на ноль и функция вернула nan, то присваиваем ноль
    if isnan(r):
        r = 0
    return r

In [26]:

def topMatches(critics_dict, person, similarity=sim_pearson):
    # your code here
    # инициализируем лист оценок похожести
    scores = []
    # для каждого человека в словаре критиков, если этот человек не тот, для которого мы подбираем соответствия
    for user in critics_dict.keys():
        if person != user:
            # добавить в scores похожесть, вычисленную с помощью функции, записанной в similarity, и имя человека
            scores.append((user, similarity(critics_dict, person, user)))
    # отсортировать список по убыванию оценок
    scores.sort(key = lambda x: x[1], reverse=True)
    # возвращаем первые n человек
    return scores


In [90]:
random_user = pick_random_user(data).index[0]
print(random_user)

96874


In [91]:
matches = topMatches(critics, random_user)

C:\Users\milen\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [93]:
euclid_matches = topMatches(critics, random_user, similarity=euclid_oxana)

In [95]:
print("Pearson best films")
for match in matches[:3]:
    films = critics[match[0]]
    print(f"You perfectly match with user {match[0]} : {match[1]}, you should watch his films:")
    for film in films.keys():
     print(f"\t{movies[film]}")

print("Euclid best films")
for match in euclid_matches[:3]:
    films = critics[match[0]]
    print(f"You perfectly match with user {match[0]} : {match[1]}, you should watch his films:")
    for film in films.keys():
        print(f"\t{movies[film]}")

Pearson best films
You perfectly match with user 24607 : 1.0, you should watch his films:
	Heat
	For a Few Dollars More
	Pandora's Box
	Open Water
	Ludwig
You perfectly match with user 137740 : 1.0, you should watch his films:
	Heat
	Earth Girls Are Easy
	Cars
You perfectly match with user 161069 : 1.0, you should watch his films:
	Heat
	To Be or Not to Be
	The Projected Man
Euclid best films
You perfectly match with user 255439 : 1.0, you should watch his films:
	Heat
	The Time Machine
	All the President's Men
	The Assassination of Richard Nixon
You perfectly match with user 229439 : 1.0, you should watch his films:
	GoldenEye
	Home Alone
	The African Queen
	Say Anything...
	Spaceballs
You perfectly match with user 50323 : 1.0, you should watch his films:
	Cutthroat Island
	Houseboat
	Holy Matrimony
	Lucky Number Slevin


In [110]:
from sklearn.metrics import  precision_score, recall_score, f1_score, accuracy_score, mean_squared_error

# [0, 5] -> [0 , 1]
def scaler(value):
    return value / 5.0

# [-1, 1] -> [0, 1]
def scale_pearson(value):
    return (value + 1)/2

def report(predicted, truth, predicted_binary, truth_binary):
    presicion = precision_score(predicted_binary, truth_binary)
    recall = recall_score(predicted_binary, truth_binary)
    f1 = f1_score(predicted_binary, truth_binary)
    accuracy = accuracy_score(predicted_binary, truth_binary)
    mse = mean_squared_error(predicted, truth)
    rmse = np.sqrt(mse)
    return (presicion, recall, f1, accuracy, mse, rmse)

# сравниваем оценку на фильм colaborative filtration и саму оценку пользователя для этого фильма
def recommendation_correctness(critics_recommendations, critics_scores, pearson=False):
    predicted = []
    predicted_binary = []
    truth = []
    truth_binary = []
    for predictions, critic_score in zip(critics_recommendations, critics_scores):
        for prediction in predictions:
            films = critics[prediction[0]]
            for film_id in films.keys():
                if film_id in critic_score.keys():
                    predicted.append(scale_pearson(prediction[1]) if pearson else prediction[1])
                    # результат нашей модели, и мы ей мало доверяем
                    predicted_binary.append( 1 if prediction[1] >= 0.95 else 0)
                    truth.append(scaler(critic_score[film_id]))
                    # оценка пользователя 4+ -> хороший фильм
                    truth_binary.append( 1 if scaler(critic_score[film_id])>= 0.8 else 0)
    score = [ v for v in report(predicted, truth, predicted_binary, truth_binary)]
    data_ = {"score": score}
    report_ = pd.DataFrame(data_, index=['presicion','recall','f1-score', 'accuracy', 'mse', 'rmse']).T
    display(report_)

In [111]:
print("Euclid correctness for 1 user")
recommendation_correctness([euclid_matches],[critics[random_user]])

Euclid correctness for 1 user


,presicion,recall,f1-score,accuracy,mse,rmse
score,0.001163,0.679422,0.002321,0.565134,0.375353,0.61266


In [112]:
print("Pierson correctness for 1 user")
recommendation_correctness([matches],[critics[random_user]], pearson=True)

Pierson correctness for 1 user


,presicion,recall,f1-score,accuracy,mse,rmse
score,0.003448,0.46074,0.006845,0.564612,0.059703,0.244341


In [185]:
random_critics = []
recommendations = []
films = []
for _ in range(10):
    # выбор уникального критика
    random_critic = pick_random_user(data).index[0]
    while random_critic in random_critics:
        random_critic = pick_random_user(data).index[0]
    random_critics.append(random_critic)
    recommendations.append(topMatches(critics, random_critic, similarity=euclid_oxana))
    films.append(critics[random_user])
print("Euclid correctness for 10 users")
recommendation_correctness(recommendations,films)

Euclid correctness for 10 users


,presicion,recall,f1-score,accuracy,mse,rmse
score,0.001001,0.534683,0.001999,0.564928,0.3734,0.611065


In [ ]:
random_critics = []
recommendations = []
films = []
for _ in range(10):
    # выбор уникального критика
    random_critic = pick_random_user(data).index[0]
    while random_critic in random_critics:
        random_critic = pick_random_user(data).index[0]
    random_critics.append(random_critic)
    recommendations.append(topMatches(critics, random_critic))
    films.append(critics[random_user])
print("Pierson correctness for 10 users")
recommendation_correctness(recommendations,films, pearson=True)

C:\Users\milen\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
C:\Users\milen\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
C:\Users\milen\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
C:\Users\milen\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
C:\Users\milen\A

Pierson correctness for 10 users


,presicion,recall,f1-score,accuracy,mse,rmse
score,0.003252,0.507722,0.006462,0.564915,0.056557,0.237817


## Задание 6

Используйте метод Term Frequency Inverse Document Frequency (TF-IDF), чтобы отфильтровать фильмы, похожие (используйте для этого косинусное расстояние) на те, которые пользователь высоко оценил.
Оцените качество такой рекомендации.

In [187]:
%%capture
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [234]:
tfidf_data = metadata.copy()

**Уберем знаки препинания**

In [235]:
def remove_punctuation(sentence):
    for ch in string.punctuation:
        sentence = sentence.replace(ch,"")
    return sentence.lower()
tfidf_data['overview'] = tfidf_data.apply(lambda row: remove_punctuation(row['overview']), axis=1)
display(tfidf_data.head())

,id,imdb_id,overview,title
0,862,tt0114709,led by woody andys toys live happily in his ro...,Toy Story
1,8844,tt0113497,when siblings judy and peter discover an encha...,Jumanji
2,15602,tt0113228,a family wedding reignites the ancient feud be...,Grumpier Old Men
3,31357,tt0114885,cheated on mistreated and stepped on the women...,Waiting to Exhale
4,11862,tt0113041,just when george banks has recovered from his ...,Father of the Bride Part II


In [236]:
from nltk.tokenize import word_tokenize
tfidf_data['overview'] = tfidf_data.apply(lambda row: word_tokenize(row['overview'], language='english'), axis=1)
display(tfidf_data.head())

,id,imdb_id,overview,title
0,862,tt0114709,"[led, by, woody, andys, toys, live, happily, i...",Toy Story
1,8844,tt0113497,"[when, siblings, judy, and, peter, discover, a...",Jumanji
2,15602,tt0113228,"[a, family, wedding, reignites, the, ancient, ...",Grumpier Old Men
3,31357,tt0114885,"[cheated, on, mistreated, and, stepped, on, th...",Waiting to Exhale
4,11862,tt0113041,"[just, when, george, banks, has, recovered, fr...",Father of the Bride Part II


**Лемматизация**

In [237]:
%%capture
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer

In [238]:
# проверка
morph = MorphAnalyzer()
print(morph.parse('мыши')[0].normal_form)

мышь


In [239]:
def lemmatize(sentence):
    return [morph.parse(w)[0].normal_form for w in sentence]
tfidf_data['overview'] = tfidf_data.apply(lambda row: lemmatize(row['overview']), axis=1)
display(tfidf_data.head())

,id,imdb_id,overview,title
0,862,tt0114709,"[led, by, woody, andys, toys, live, happily, i...",Toy Story
1,8844,tt0113497,"[when, siblings, judy, and, peter, discover, a...",Jumanji
2,15602,tt0113228,"[a, family, wedding, reignites, the, ancient, ...",Grumpier Old Men
3,31357,tt0114885,"[cheated, on, mistreated, and, stepped, on, th...",Waiting to Exhale
4,11862,tt0113041,"[just, when, george, banks, has, recovered, fr...",Father of the Bride Part II


In [242]:
def convert_to_string(sentence):
    return ' '.join(sentence)
tfidf_data['overview'] = tfidf_data.apply(lambda row: convert_to_string(row['overview']), axis=1)
display(tfidf_data.head())

,id,imdb_id,overview,title
0,862,tt0114709,led by woody andys toys live happily in his ro...,Toy Story
1,8844,tt0113497,when siblings judy and peter discover an encha...,Jumanji
2,15602,tt0113228,a family wedding reignites the ancient feud be...,Grumpier Old Men
3,31357,tt0114885,cheated on mistreated and stepped on the women...,Waiting to Exhale
4,11862,tt0113041,just when george banks has recovered from his ...,Father of the Bride Part II


In [240]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words="english")

In [243]:
tfidf_matrix = tf.fit_transform(tfidf_data['overview'])
tfidf_matrix.shape

(43856, 1055410)

In [244]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix)

In [245]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=tfidf_data['title'], columns=tfidf_data['title'])

In [246]:
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

title,Pure,Blood Diner,Bleach: The DiamondDust Rebellion,Vieni avanti cretino,"When You Comin' Back, Red Ryder?"
title,,,,,
Bedrooms and Hallways,0.009968,0.003877,0.000000,0.0,0.000000
The Admirable Crichton,0.000000,0.000000,0.000000,0.0,0.000000
The Lost Weekend,0.000000,0.000000,0.000000,0.0,0.000000
Car Trouble,0.000000,0.007397,0.000000,0.0,0.010494
Red Eagle,0.000000,0.005696,0.004426,0.0,0.008387
Walkabout,0.006559,0.000000,0.000000,0.0,0.000000
Side Street,0.003650,0.002755,0.000000,0.0,0.000000
"Dr. Black, Mr. Hyde",0.000000,0.000000,0.000000,0.0,0.000000
Atomic Age,0.002968,0.004774,0.000000,0.0,0.000000


In [300]:
def build_recommendations(movie_title, similarity_df, output_df, k=10):
    # взять k самых больших значений из матрицы похожести по movie_title
    ix = similarity_df.loc[:,movie_title].to_numpy().argpartition(range(-1,-k,-1))
    # взять k + 1 названия колонок по полученным на предыдущем шаге id колонок
    closest = similarity_df.columns[ix[-1:-(k+2):-1]]
    closest = closest.drop(movie_title, errors='ignore')
    return pd.DataFrame(closest).merge(output_df).head(k)

In [252]:
build_recommendations('Charlie: The Life and Art of Charles Chaplin', cosine_sim_df, tfidf_data[['title', 'overview']])

,title,overview
0,"Jonathan Rosenbaum, Present",a portrait of the influential american film cr...
1,The Last Ferry,directed by
2,Life Itself,life itself recounts the surprising and entert...
3,City Lights,city lights is the first silent film that char...
4,Fracchia The Human Beast,the worlds greatest criminal and the world gre...
5,Mister Lonely,in paris a young american who works as a micha...
6,Role Play: Women on Fassbinder,interviews with four women who were important ...
7,Angele,angèle is a 1934 french drama film directed pr...
8,Film Geek,the story of scotty pelk a socially inept vide...
9,The Wall,a woman inexplicably finds herself cut off fro...


In [253]:
build_recommendations('Jumanji', cosine_sim_df, tfidf_data[['title', 'overview']])

,title,overview
0,Word Wars,the classic board game scrabble has been popul...
1,The Bar,a group of strangers are trapped inside a bar
2,Table No. 21,a couple live a mediocre life and are thrilled...
3,Quintet,during a future ice age dying humanity occupie...
4,Liar Game: Reborn,to exact revenge the liar game office is reviv...
5,Le Pont du Nord,marie is just out from prison when she runs in...
6,Doctor Who: Last Christmas,the doctor and clara face their last christmas...
7,Not Safe for Work,an office worker is trapped inside the buildin...
8,The Ouija Exorcism,in 1985 a celebrated exorcist trapped a horrif...
9,Rhinoceros,originally an absurdist play by eugene ionesco...


In [329]:
# сравниваем оценку на фильм colaborative filtration и саму оценку пользователя для этого фильма
def tfidf_recommendation_correctness(user_count):
    # взять рандомного юзера у которого ревью больше 30
    # выбираем фильмы у которого он поставил оценку больше =  4 =  N, // 5 фильмов
    # если их меньше 2, он нам не подходит
    # выбираем один из любимых и ищем похожие фильмы N - 1 // находим 4 фильма
    # совпали с теми  // 4/4
    random_users = []
    while len(random_users) < user_count:
        user = pick_random_user(data, 666).index[0]
        random_users.append(user)
        films = critics[user]
        films_to_find = []
        for film_id in films.keys():
            if films[film_id] >= 4:
                films_to_find.append(film_id)
        if  len(films_to_find) < 2:
            break
        film_id = films_to_find[0]
        film_name = metadata.loc[metadata['id'] == film_id]['title'].tolist()[0]
        recommendations_found = build_recommendations(film_name, cosine_sim_df, tfidf_data[['title','id']],
                                                       k=len(films_to_find) - 1)['id'].tolist()

        hits = 0
        for recommendation_id in recommendations_found:
            if recommendation_id in films_to_find:
                hits += 1
        print(f"films guessed {hits}")

In [330]:
tfidf_recommendation_correctness(1)

films guessed 8
